In [2]:
!pip install pyspark grpcio grpcio-status pandas pyarrow catboost mlflow boto3 bentoml
!pip install --extra-index-url https://nexus.baltinfocom.ru/repository/pypi/simple stat_helper==0.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 2.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.0.0
    Uninstalling pyarrow-16.0.0:
      Successfully uninstalled pyarrow-16.0.0
Looking in indexes: https://pypi.org/simple, https://nexus.baltinfocom.ru/repository/pypi/simple


In [13]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession

In [14]:
spark = (
    SparkSession
    .builder
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0")
    .config("spark.executor.memory", '64g')
    .config('spark.executor.cores', '10')
    .config('spark.cores.max', '10')
    .config("spark.driver.memory",'64g')
    .getOrCreate()
)

In [15]:
POSTGRES_URL = 'jdbc:postgresql://192.168.2.110:5434/postgres'
POSTGRES_USER = ""
POSTGRES_PASSWORD = ""
SCHEMA_NAME = "sc_billing_198"
NUMBER_TUPLES_TABLE_NAME = 's_number_tuples'
SESSION_TABLES = ['s_sessions_2012_10_01', 's_sessions_2012_11_01']
MIN_NUMBER_OF_SESSIONS = 10
BATCH_SIZE = 1000

In [16]:
query = f'''
    select number_tuple_id
    from {SCHEMA_NAME}.{NUMBER_TUPLES_TABLE_NAME}
    where number_of_sessions >= {MIN_NUMBER_OF_SESSIONS}
    order by number_tuple_id
    limit {BATCH_SIZE}
'''

user_df = (
        spark.read 
        .format("jdbc") 
        .option("driver", "org.postgresql.Driver") 
        .option("url", POSTGRES_URL)
        .option("user", POSTGRES_USER) 
        .option("password", POSTGRES_PASSWORD)
        .option("query", query)
        .load()
)

In [17]:
first_id, last_id = user_df.head(1)[0].number_tuple_id, user_df.tail(1)[0].number_tuple_id

In [18]:
user_session_df = None

for table in SESSION_TABLES:
    query = f'''
        select distinct
            left_number_tuple_id as number_tuple_id,
            right_number_tuple_id as interlocutor_number_tuple_id,
            true as is_outgoing,
            s_dt,
            s_dur
        from {SCHEMA_NAME}.{table}
        where {first_id} <= left_number_tuple_id and left_number_tuple_id <= {last_id} 
        union all
        select distinct
            right_number_tuple_id as number_tuple_id,
            left_number_tuple_id as interlocutor_number_tuple_id,
            false as is_outgoing,
            s_dt,
            s_dur
        from {SCHEMA_NAME}.{table}
        where {first_id} <= right_number_tuple_id and right_number_tuple_id <= {last_id}
    '''
    df = (
        spark.read 
        .format("jdbc") 
        .option("driver", "org.postgresql.Driver") 
        .option("url", POSTGRES_URL)
        .option("user", POSTGRES_USER) 
        .option("password", POSTGRES_PASSWORD)
        .option("query", query)
        .load()
    )
    if user_session_df is None:
        user_session_df = df
    else:
        user_session_df.union(df)

In [19]:
from stat_helper import calc_stat

user_stat_df = calc_stat(user_session_df)

Добавление лейблов 

In [20]:
user_stat_df = user_stat_df.withColumn('sex', f.when(f.rand() > 0.5, 1).otherwise(0))
user_stat_df.printSchema()

root
 |-- number_tuple_id: integer (nullable = true)
 |-- perc_85_not_null_out_s_duration: double (nullable = true)
 |-- perc_85_in_s_duration: double (nullable = true)
 |-- percent_not_null_g_30_s_sessions: double (nullable = true)
 |-- percent_not_null_out_g_15_s_sessions: double (nullable = true)
 |-- max_s_duration: integer (nullable = true)
 |-- percent_not_null_out_g_60_s_sessions: double (nullable = true)
 |-- percent_not_null_in_g_15_s_sessions: double (nullable = true)
 |-- percent_in_l_15_s_sessions: double (nullable = true)
 |-- perc_85_s_duration: double (nullable = true)
 |-- percent_out_l_15_s_sessions: double (nullable = true)
 |-- median_not_null_in_s_duration: double (nullable = true)
 |-- percent_not_null_in_g_30_s_sessions: double (nullable = true)
 |-- perc_85_out_s_duration: double (nullable = true)
 |-- median_in_s_duration: double (nullable = true)
 |-- max_out_s_duration: integer (nullable = true)
 |-- percent_g_15_s_sessions: double (nullable = true)
 |-- perce

Обучение модели

In [21]:
import catboost as cb
import catboost.utils as cbu
import numpy as np
from catboost import CatBoostClassifier, Pool

In [22]:
from sklearn.model_selection import train_test_split

pd_df = user_stat_df.toPandas()

train_df, test_df = train_test_split(pd_df, test_size=0.1)

label = 'sex'
ignore = ['number_tuple_id'] + [label]
feature_names = [col for col in user_stat_df.columns if col not in ignore]
train_pool = Pool(train_df[feature_names], train_df[label])
test_pool = Pool(test_df[feature_names], test_df[label])

In [23]:
hyperparameters = {
    'iterations': 500,
    'depth': 3,
    'learning_rate': 1,
}
model = CatBoostClassifier(**hyperparameters)

In [24]:
model.fit(train_pool)

0:	learn: 0.6845140	total: 51.9ms	remaining: 25.9s
1:	learn: 0.6809518	total: 54ms	remaining: 13.4s
2:	learn: 0.6676632	total: 56.2ms	remaining: 9.31s
3:	learn: 0.6623457	total: 58ms	remaining: 7.19s
4:	learn: 0.6563648	total: 59.7ms	remaining: 5.91s
5:	learn: 0.6429177	total: 65.8ms	remaining: 5.42s
6:	learn: 0.6365169	total: 67.6ms	remaining: 4.76s
7:	learn: 0.6231201	total: 69.9ms	remaining: 4.3s
8:	learn: 0.6093108	total: 71.9ms	remaining: 3.92s
9:	learn: 0.5960908	total: 74.6ms	remaining: 3.66s
10:	learn: 0.5860015	total: 76.2ms	remaining: 3.39s
11:	learn: 0.5792428	total: 77.7ms	remaining: 3.16s
12:	learn: 0.5655275	total: 79.5ms	remaining: 2.98s
13:	learn: 0.5534810	total: 82.4ms	remaining: 2.86s
14:	learn: 0.5460735	total: 84ms	remaining: 2.72s
15:	learn: 0.5392261	total: 85.6ms	remaining: 2.59s
16:	learn: 0.5261798	total: 87.1ms	remaining: 2.48s
17:	learn: 0.5176371	total: 88.9ms	remaining: 2.38s
18:	learn: 0.5078886	total: 90.5ms	remaining: 2.29s
19:	learn: 0.4929998	total: 9

In [25]:
metric_names = ['AUC', 'Logloss', 'CrossEntropy', 'F1']
metrics = {
    name: metric[-1] for name, metric in model.eval_metrics(test_pool, metric_names, eval_period=500).items()
}
metrics['Accuracy'] = model.score(test_df[feature_names], test_df[label])

### Feature selection

In [55]:
SCORE_THRESHOLD = 0.3

In [56]:
selected_features = [feature for score, feature in zip(model.get_feature_importance(), feature_names) if score > SCORE_THRESHOLD]

len(selected_features), len(feature_names)

(79, 138)

In [57]:
selected_user_stat_df = calc_stat(user_session_df)

In [58]:
selected_user_stat_df.printSchema()

root
 |-- number_tuple_id: integer (nullable = true)
 |-- perc_85_not_null_out_s_duration: double (nullable = true)
 |-- perc_85_in_s_duration: double (nullable = true)
 |-- percent_not_null_g_30_s_sessions: double (nullable = true)
 |-- percent_not_null_out_g_15_s_sessions: double (nullable = true)
 |-- max_s_duration: integer (nullable = true)
 |-- percent_not_null_out_g_60_s_sessions: double (nullable = true)
 |-- percent_not_null_in_g_15_s_sessions: double (nullable = true)
 |-- percent_in_l_15_s_sessions: double (nullable = true)
 |-- perc_85_s_duration: double (nullable = true)
 |-- percent_out_l_15_s_sessions: double (nullable = true)
 |-- median_not_null_in_s_duration: double (nullable = true)
 |-- percent_not_null_in_g_30_s_sessions: double (nullable = true)
 |-- perc_85_out_s_duration: double (nullable = true)
 |-- median_in_s_duration: double (nullable = true)
 |-- max_out_s_duration: integer (nullable = true)
 |-- percent_g_15_s_sessions: double (nullable = true)
 |-- perce

In [90]:
train_df = selected_user_stat_df.withColumn('sex', f.when(f.rand() > 0.5, 1).otherwise(0)).toPandas()

In [92]:
train_pool = Pool(train_df[selected_features], train_df[label])

In [93]:
model.fit(train_pool)

0:	learn: 0.6852066	total: 2.85ms	remaining: 1.42s
1:	learn: 0.6738216	total: 4.61ms	remaining: 1.15s
2:	learn: 0.6637438	total: 6.08ms	remaining: 1.01s
3:	learn: 0.6568653	total: 7.55ms	remaining: 936ms
4:	learn: 0.6517533	total: 9.05ms	remaining: 896ms
5:	learn: 0.6465740	total: 10.5ms	remaining: 867ms
6:	learn: 0.6405012	total: 11.9ms	remaining: 842ms
7:	learn: 0.6307282	total: 13.6ms	remaining: 836ms
8:	learn: 0.6167215	total: 15.1ms	remaining: 822ms
9:	learn: 0.6100602	total: 16.5ms	remaining: 808ms
10:	learn: 0.5969324	total: 18.2ms	remaining: 811ms
11:	learn: 0.5805341	total: 19.7ms	remaining: 800ms
12:	learn: 0.5678678	total: 21.1ms	remaining: 790ms
13:	learn: 0.5559685	total: 22.5ms	remaining: 782ms
14:	learn: 0.5474287	total: 24ms	remaining: 775ms
15:	learn: 0.5354601	total: 25.4ms	remaining: 768ms
16:	learn: 0.5257092	total: 27.6ms	remaining: 783ms
17:	learn: 0.5127475	total: 29ms	remaining: 777ms
18:	learn: 0.5048907	total: 30.4ms	remaining: 770ms
19:	learn: 0.4960294	total

### Загрузка модели в mlflow

In [94]:
import os
import mlflow

os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://node04.bic2020.bic:9200"

In [95]:
MLFLOW_URL = 'http://node04.bic2020.bic:5000'
MLFLOW_MODEL_NAME = 'catboost-stat-sex-predict-model'

In [96]:
with mlflow.start_run():
    mlflow.set_tracking_uri(uri=MLFLOW_URL)
    mlflow.set_experiment("Sex prediction")
    mlflow.log_param("features", selected_features)
    info = mlflow.catboost.log_model(
        cb_model=model,
        artifact_path=MLFLOW_MODEL_NAME,
        registered_model_name=MLFLOW_MODEL_NAME,
    )
    mlflow.log_metrics(metrics)

Registered model 'catboost-stat-sex-predict-model' already exists. Creating a new version of this model...
2024/05/16 07:28:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: catboost-stat-sex-predict-model, version 9
Created version '9' of model 'catboost-stat-sex-predict-model'.


### Запросы к BentoML-сервису

In [97]:
import bentoml

In [98]:
BENTOML_ENDPOINT_URL = 'http://node04.bic2020.bic:3000'
MLFLOW_MODEL_NAME = 'catboost-stat-sex-predict-model'
MLFLOW_MODEL_ALIAS = 'champion'

In [99]:
data = train_df[selected_features].to_dict('list')

with bentoml.SyncHTTPClient(BENTOML_ENDPOINT_URL) as client:
    predicted_labels = client.predict(
        data=data,
        model_name=MLFLOW_MODEL_NAME,
        model_alias=MLFLOW_MODEL_ALIAS,
    )

predicted_labels

[1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
